# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**10/06/2020 - C11 S7**

# Ingeniería de Características y Algoritmos de Aprendizaje Automático

La práctica de la ciencia de datos se fundamenta en la obtención, exploración, limpieza, tratamiento y transformación de la información contenida en distintas fuentes de datos. Tales procedimientos constituyen gran parte del desarrollo de proyectos basados datos y su finalidad es la facilitar la obtención de conocimiento. Este último paso se lleva a cabo por medio de sistemas de aprendizaje automático. En general esto se puede resumir en las siguientes etapas:

1. **Obtención de la información**: Corresponde a la recolección de datos referentes al fenómeno de interés. Tal información puede ser tanto estructurada como no estructurada.

2. **Preprocesamiento y exploración**: Una vez reunida una porción de información significativa, se procede a explorarla y procesarla. En este apartado se aplican métodos de reducción de dimensionalidad, escalamiento, muestreo (conjuntos de entrenamiento y test), selección de características y transformación. 

3. **Aprendizaje**: Cuando los datos han sido estudiados y transformados de buena manera, se procede a la aplicación de modelos de aprendizaje de máquinas. Acá se estudian métricas de rendimiento y se optimizan hiperparámetros. 

4. **Evaluación**: Seleccionado un modelo de optimo, se generan esquemas de evaluación sobre conjuntos de datos no observados pero representativos del fenómeno estudiado, esto permite adquirir nociones sobre el comportamiento del sistema modelado fuera del conjunto de entrenamiento. 

5. **Producción**: Finalmente se genera un entorno en cual el modelo obtenido es utilizado para la obtención de información, elaboración de predicciones, clasificaciones o perfilamientos en función de la materia estudiada. 

Estos procesos son interdependientes y no presentan un orden lineal, más bien, se entrelazan circulando de manera natural entre los pasos 2 y 4. También se relacionan los pasos 1 y 6 a medida que llegan nuevos datos al sistema. 

Hasta el momento nos hemos centrado en las etapas de obtención, preprocesamiento y exploración de datos. El paso siguiente corresponde a un *refinamiento* de la información, con el fin de entrenar modelos de aprendizaje de alto rendimiento sobre los datos que se poseen. Este refinamiento ve su primera etapa en los procesos exploratorios (perfilamiento de datos y preprocesamiento inicial), para luego generar transformaciones y selecciones sobre las variables disponible. Dichas transformaciones y selecciones logran un mejor rendimiento en la medida que se obtienen por medio de *conocimiento del área* (*domain kwnlegde*), es decir, utilizando el conocimiento existente, con respecto al problema que se desea modelar. Sin embargo, existen ciertas técnicas *estándar* y algunas *automáticas* que permiten depurar los datos para obtener atributos y transformaciones de interés. 

En general, el proceso de refinamiento de datos se conoce como **ingeniería de características**, donde las características o *features* hacen referencia a los atributos y sus transformaciones. El proceso de ingeniería de características pasa generalmente por:

Explorar los datos y ajustar las variables disponibles, de manera tal, que se gane compatibilidad con las hipótesis de ciertos modelos de aprendizaje automático.

Posteriormente, se derivan características de interés, ya sea por medio de composición de atributos o transformaciones sobre estos. En esta parte, es una buena práctica, buscar transformaciones que hagan sentido con el problema modelado, esto no siempre se cumple (ej: una transformación polinomial sobre los datos no necesariamente es interpretable en ciertos problemas, pero puede aumentar considerablemente el rendimiento). 

Se continua, seleccionando caraterísticas, esto se puede hacer por medio de análisis estadísticos y algoritmos puntuación. Es necesario entender, que aunque en primera instancia más variables deberían llevarnos a mejores resultados, en la realidad, pueden aparecer caraterísticas que añadan ruido al sistema, ya sea por no estar relacionadas con el fenómeno estudiado o por presentarse problemas en su recolección. En tal contexto, cobra importancia la exlusión de dichas variables o de manera reciproca, la selección de aquellas que aseguren el mejor desempeño. 

Finalmente, se procede a entrenar y evaluar modelos sobre las métricas relevantes con el problema a trabajar, esta etapa se comunica constantemente con la de generación y selección de caraterísticas, pues las transformaciones que se decidan utilizar dependen en cierta medida de los modelos que se implementarán. 

A continuación se estudian ciertas técnicas de ingeniería de características estándar.

## Transformaciones de Atributos

Ya hemos estudiado técnicas de transformación por medio de preprocesamiento. A continuación se añaden y estudian nuevas metodologias además de aplicar algunas técnicas previas en la base de datos `HousePricing`. 

### Preprocesamiento inicial

Se cargan los datos y se agrupan sus columnas según tipo de dato

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('../S5/data/train.csv', index_col = 'Id')

Una de las técnicas más básicas de ingeniería de características es la **modificación de tipo de dato**, en este caso se transforman los datos reconocido como tipo `object` a tipo `str`.

In [ ]:
object_type_set = [col for col in df.columns if df[col].dtype == 'O']
df = df.astype({col:'str' for col in object_type_set})

Para trabajar de manera más sencilla con los datos se procede a agrupar los atributos por tipo de dato

In [ ]:
cat_cols = [
    'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
    'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive',
    'MiscFeature', 'SaleType', 'SaleCondition'
]

ordinal_cols = [
    'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
    'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PoolQC', 'Fence'
]


num_cols = [
    'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
    'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
    'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
    'MoSold', 'YrSold','SalePrice'
]

Se genera un *mapping* para obtener un datset multindexado

In [ ]:
mapping = [('numeric', col) for col in num_cols]
mapping.extend([('categorical', col) for col in cat_cols])
mapping.extend([('ordinal', col) for col in ordinal_cols])

df = df.reindex(columns=num_cols + cat_cols + ordinal_cols)
df.columns = pd.MultiIndex.from_tuples(mapping)

Obtenemos visualizaciones según tipo de variable

In [ ]:
def type_plotter(df, dtype, nrows=6):
    '''Permite graficar subconjuntos de variables en un dataframe multindexado
    
    Toma como argumento un dataframe de pandas df multi-indexado y un nivel 
    de multi - indice asociado a un subconjunto de columnas. Obtiene graficos
    univariados asociados a dicho subconjunto de varaibles.
    
    Args:
    -----
    
    df: DataFrame
        El conjunto de datos a explorar.
    
    dtype: String
        El nombre del nivel a visualizar
    
    nrwos: int
        La cantidad de filas a generar en la matriz de graficos
    
    Returns: 
    -------        
        None
        Se genera una visualizacion de matplotlib.
    '''

    cols = df[dtype].shape[1] // nrows
    resto = df[dtype].shape[1] % nrows

    fig, ax = plt.subplots(nrows=nrows + 1, ncols=cols, figsize=[17, 17])

    # Se remueve el resto de plots
    list(map(lambda a: a.remove(), ax[-1, -(cols - resto):]))

    fig.tight_layout()
    # Se define un titulo y su ubicacion
    fig.suptitle('Distribuciones Univariadas typo: ' + dtype,
                 fontsize=20,
                 x=0.5,
                 y=1.05)
    '''
    Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
    distinto en funcion del tipo de dato.

    '''
    df_cols = df[dtype].columns
    for axis, col in zip(ax.flatten(), df_cols):
        try:
            # Graficos para datos numericos
            sns.distplot(df[(dtype, col)], ax=axis, rug=True)

        except RuntimeError:
            sns.distplot(df[(dtype, col)], ax=axis, rug=True, kde=False)

        except ValueError:
            sns.countplot(df[(dtype, col)], ax=axis)

        axis.set_xlabel(col, fontsize=15)

    # Se ajusta el espaciado interno entre subplots
    w, h = (.4, .4)
    plt.subplots_adjust(wspace=w, hspace=h)

Generamos visualizaciones para las variables numéricas

In [ ]:
type_plotter(df,'numeric')

El manejo de valores faltantes corresponde a una técnica de preprocesamiento ya estudiada y previa a las transformaciones de atributos buscadas en ingeniería de características. Por tal motivo, se hace un tratamiento rápido de los valores faltantes, sin ahondar en detalles. En este apartado, se reemplazan las variables con texto 'nan' por su objeto equivalente

In [ ]:
df.replace('nan',np.nan, inplace = True) 

Se observa la distribución porcentual de los valores perdidos

In [ ]:
def find_missing_per(df):
    na_percent = (df.isnull().sum()/len(df))[(df.isnull().sum()/len(df))>0].sort_values(ascending=False)

    return pd.DataFrame({'Missing Percentage':na_percent*100})

find_missing_per(df)

Se genera una función auxiliar para tratar con multi indices

In [ ]:
def indexer(cols, t_c = df.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''
    
    set_to_tuple = set(*[cols])

    tuples = [
        i for i in t_c if set_to_tuple.intersection(set(i))
    ]
    
    return tuples

Se llenan los valores faltantes

In [ ]:
# Se transforma nan -> None
to_none = [
    'FireplaceQu', 'Fence', 'Alley', 'MiscFeature', 'PoolQC', 'MSSubClass',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType'
]

# Fill mapper
to_fill = {key: 'None' for key in indexer(to_none)}

# Se transforma nan -> 0
to_0 = [
    'GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'
]
to_fill.update({key: 0 for key in indexer(to_0)})

# Se llenan los valores faltantes
df.fillna(to_fill, inplace=True)

Se estudia el dataset luego de llenar los valores anteriores

In [ ]:
find_missing_per(df)

En este caso se tiene que las variables `LotFrontage` y `Electrical` presentan valores faltantes. Para la variable `Electrical` esto representa una cantidad muy baja de observaciones, por lo que simplemente se pueden eliminar, mientras que para la variable `LotFrontage` se escoge llenar los valores faltantes por medio de una subagrupación en función de la variable 'Neighborhood', con esto, se asigna el valor de la mediana de 'LotFrontage' en para cada subgrupo. 

In [ ]:
group_median = lambda x: x.fillna(x.median())

df.dropna(axis=0, subset=indexer(['Electrical']), inplace= True)

df[indexer(['LotFrontage'])] = df.groupby(indexer(['Neighborhood']))[indexer(['LotFrontage'])].transform(
    group_median)

finalmente no se tienen valores faltantes

In [ ]:
len(find_missing_per(df))

El siguiente paso es trabajar con los valores anómalos. Nuevamente, este paso es previo a la transformación de atributos por lo que no se estudia en profundidad.

In [ ]:
varlist = [
    'GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'MasVnrArea', 'GarageArea',
    'TotRmsAbvGrd'
]

from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN


def scatter_vs_price(df=df,
                     varlist=varlist,
                     nrows=3,
                     db_min_samples=5,
                     db_eps=0.99):
    
    '''Grafica una lista de variables contra SalePrice.
    
    Acepta como argumento una lista de variables, cada una de estas es 
    escalada de manera robusta, posteriormente se entrena una aglomeracion
    usando dbscan para resaltar los posible outliers.
    
    Parameters:
    -----------
    
    df: Pandas DataFrame
        El conjunto de datos de HousePricing
    
    varlist : List
              Lista de variables a estudiar
    
    nrows: int
           Cantidad de filas a graficar (argumento de subplots)
    
    db_min, db_eps: int, float
                    Hiperparametros de DBSCAN
    
    Returns:
    --------
        Visualizacion: None
                       Visualizacion de las variables estudiadas contra el precio
        
        Outliers: Dict
                  Diccionario de posibles outliers
    '''
    outliers = dict()
    
    ncols = len(varlist) // nrows
    resto = len(varlist) % nrows

    fig, ax = plt.subplots(nrows + 1, ncols, figsize=[13, 10])
    # Se borran las axis innecesarias
    for a in ax[-1, -(ncols - resto):]:
        a.remove()
    ax = ax.ravel()

    for a, var in zip(ax, varlist):
        
        X = df[indexer([var, 'SalePrice'])]
        scaler = RobustScaler()
        X = scaler.fit_transform(X)

        outlier_detection = DBSCAN(min_samples=db_min_samples, eps=db_eps)
        C = outlier_detection.fit_predict(X)

        a.scatter(x=X[:, 0], y=X[:, 1], c=C, alpha=0.5)

        #a.axvline(x=4600, color='r', linestyle='-')
        title = 'Scaled' + var + '- Price scatter plot'
        a.set_title(title, fontsize=15, weight='bold')
        outliers.update({var:C})
        
    fig.tight_layout()
    return outliers

Se estudian las variables seleccionadas

In [ ]:
outliers = scatter_vs_price()

Por simplicidad, se eliminan los valores anómalos.

In [ ]:
# Se recolectan los indices de valores anomalos
outlier_idx = pd.Index({})
for key, val in outliers.items():

    outlier_locs = [*map(bool, val)]
    columns = indexer([key])

    outlier_idx = outlier_idx.union(df.loc[outlier_locs, columns].index)
    
# Se eliminan las filas con valores anomalos
df.drop(index=outlier_idx, axis=0, inplace=True)

Si bien, luego de este preprocesamiento se puede trabajar en transformaciones sobre los datos, se debe recordar, que las etapas de preprocesamiento y manejo de caraterísticas se comunican constantemente, generando ciclos en el flujo de trabajo. 

### Manejo de Carácteristicas Básico


A continuación se procede a producir características relevantes, una primera aproximación a este procedimiento es comenzar por introducir transformaciones sobre las variables y modelar la interacción entre estas. En este apartado, es necesario discutir el término de **Data Leakage**. 

**Data Leakage** hace referencia a la *fuga* de información entre conjuntos destinados para entrenamiento y test. Si se produce esta fuga, significa que se esta utilizando información no valida en procesos de entrenamiento, lo cual genera modelos sobre optimistas en sus métricas de evaluación. En la práctica, un modelo con fuga de información puede ser incluso inútil en producción. Para evitar este fenómeno, basta realizar las transformaciones de datos pertinentes dentro de los procesos de validación y entrenamiento. Otra buena práctica en este aspecto, es generar un conjunto de validación adicional para comprobar métricas de rendimiento luego de realizar selección de modelo.

A modo de ejemplo: si en un problema de modelamiento se normalizan ciertas variables utilizando la información total del dataset y luego se procede a generar particiones de entrenamiento y validación, entonces los elementos ubicados en los conjuntos de validación habrán aportado información al proceso de normalización pues influyeron en los calculos de media y desviación estándar. Esto puede mejorar de cierta manera el rendimiento en predicción, dando como resultado un modelo sesgado.

Lo primero que se hará es generar un conjunto de validación utilizando el 10% de los datos, este conjunto se utiliza al final del proceso de selección de modelos para verificar la capacidad de generalización.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size = .1)

A continuación, se procede a estudiar ciertos atributos del conjunto de datos

In [ ]:
f, ax = plt.subplots(figsize=(8, 7))

sns.set_style("white")
sns.set_color_codes(palette='deep')


sns.distplot(df_train['numeric']['SalePrice'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frecuancia")
ax.set(xlabel="SalePrice")
ax.set(title="Distribucion de SalePrice")

sns.despine(trim=True, left=True)
plt.show()

Vemos que tiene similitud a una distribución normal, para ello se hace un test de normalidad

In [ ]:
from scipy import stats

def norm_test(data = df_train['numeric']['SalePrice']):

    k, p = stats.normaltest(data)

    alpha = 1e-3
    print("p =",p)

    if p < alpha: 
        print("Se puede rechazar la hipotesis nula")
    else:
        print("No se puede rechazar la hipotesis nula")

In [ ]:
norm_test()

Luego la variable `SalePrice` no se comporta como una normal. En estos casos. Una técnica de manejo de características básica es la transformación de Box-Cox (ya vista), Podemos transformar esta variable en función de tal método

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Se utiliza una funcion auxiliar para transformar la serie
to_array = lambda pd_series: pd_series.values.reshape(-1, 1)
y = to_array(df_train['numeric']['SalePrice'])

transformer_bc = PowerTransformer(method='box-cox')

y = transformer_bc.fit_transform(y)

# Se aplica el test de normalidad a la variable de interes
norm_test(y)

Luego la transformación de Box - Cox permite normalizar de manera sencilla la distribución de la variable `SalePrice`. En general, todas las técnicas de preprocesamiento vistas anteriormente (escalar, normalizar,códificación dummy, etc...) entran en el manejo de básico de características. 

**Ejercicio**

1. Visualice las variables numéricas, calcule su asimetría estadística (skewness) por medio del método `.skew()`. Defina un valor umbral (ej: 0.6). Luego aplique una transformación de potencia sobre las variables más asimétricas. 

**Obs**: Lo anterior permite normalizar de manera más robusta aquellas variables que por naturaleza se alejan de ser normales.

Otra manera de generar características de interés es por medio de interacciones. En este caso se hace uso del conocimiento del área que se trabaja. 

Por ejemplo, las variables `TotalBsmtSF`, `1stFlrSF` y `2ndFlrSF` representan unidades de área en pies cuadrados para el subterraneo, primer y segundo piso. Con esta información se puede crear la variable `TotalSF` que representa la superficie total de una vivienda

In [ ]:
df_train.loc[:,('numeric','TotalSF')] = df_train['numeric']['TotalBsmtSF'] + df_train['numeric']['1stFlrSF'] + df_train[
    'numeric']['2ndFlrSF']

Se puede también general la variable `YearsSinceRem` que permite codificar la cantidad de años que han pasado desde la remodelación de una vivienda

In [ ]:
df_train.loc[:,('numeric','YearsSinceRemodel')] = df_train['numeric']['YrSold'] - df_train['numeric']['YearRemodAdd']

De la misma manera, se puede agregar la variable `TotalQual` que representa el puntaje total de calidad asociado a una vivienda

In [ ]:
df_train.loc[:,('numeric','TotalQual')]  = df_train['numeric']['OverallQual'] + df_train['numeric']['OverallCond']

En general, se pueden aplicar transformaciones exponenciales (como en el caso de Box-Cox), lineales (como los dos casos anteriores) o de cualquier otro tipo (trigonometricas, polinomiales) en función del tipo de problema que se trabaje, siempre y cuando la transformación tenga un trasfondo en el fenómeno.

**Ejercicio**

Otra forma de inducir interacciones es por medio de variables dummy, en este caso se procede a generar codificaciones one hot. En este contexto, si se posee una variable codificada de esta forma y se multiplica por otra, ocurre que la nueva variable generada modela la interesección de eventos entre esas variables. 

1. Visualice las variables categóricas, genere un esquema de codificación y modele interacciones entre variables. Cuantifique estadísticamente el efecto de las variables de interacción con la variable de respuesta. 

## Selección de características 

En conjunción con el estudio, perfilamiento y aplicación de transformaciones iniciales a los datos, se hace necesario utilizar técnicas de selección de característcas. Estas permiten mejorar métricas de rendimiento en predicción. 

Las técnicas de selección de característcas se pueden clasificar como **no supervisadas** que no usan la variable de respuesta (análisis de correlaciones en contextos de multicolinealidad). Otra categroría son las técnicas **supervisadas** , estas hacen uso de la variable de respuesta, y pueden separarse en técnicas *envolventes* o *wrappers*, las cuales seleccionan variables en función de los resultados provistos por algoritmos de aprendizaje. Se observan también los métodos de *filtrado*, que seleccionan subconjuntos de variables en función de ciertos estadísticos, valores umbrales o scoring  (pueden ser no supervisados). Además, se encuentran los métodos de selección supervisada *implícitos*, estos corresponden a metodologías naturales dentro de ciertos algoritmos de aprendizaje (ej: selección de variables por medio de regresión LASSO). Por último se pueden utilizar técnicas de **reducción de dimensionalidad** que permiten proyectar los datos input a dimensiones más bajas. 

Sci-kit Learn provee de la API `sklearn.feature_selection ` que proporciona métodos de ayuda en este apartado.

A continuación se estudian métodos de **selección de caraterísticas univariadas**.

### Umbral de varianza 

Este método permite seleccionar variables por *filtrado* no supervisado y es un acercamiento simple que se basa en la premisa de que aquellas características con muy poca varianza no aportan información al modelo. 

**Ejemplo**

Se observan las características ordinales del conjunto de entrenamiento


In [ ]:
type_plotter(df_train,dtype='ordinal')

En este caso se observa que las variables `Functional` y `PoolQc` poseen una baja varianza (entre otras variables). Se importa el método de seleccion de caraterísticas por medio de umbral de varianza y se estudia su efecto en el conjunto de entrenamiento, para ello, es necesario codificar las variables estudiadas.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

Se generan pipelines sobre las variables ordinales

In [ ]:
# Adquieren las categorias de cada variable
ordinal_cat = [['Reg', 'IR1', 'IR2', 'IR3'],
               ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'], ['Gtl', 'Mod', 'Sev'],
               ['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'No', 'Mn', 'Av', 'Gd'],
               ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
               ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
               ['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
               ['None', 'Po', 'Fa', 'TA', 'Gd',
                'Ex'], ['None', 'Unf', 'RFn', 'Fin'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']]

ord_pipe = Pipeline(
    steps=[('ordinal', OrdinalEncoder(categories = ordinal_cat))])

In [ ]:
X_train = df_train.drop(('numeric','SalePrice'), axis=1).dropna(how = 'all').copy()

# Variable dependiente
y_train = df_train['numeric']['SalePrice'].copy()

# Se preparan los datos
X_prep = ord_pipe.fit_transform(X_train['ordinal'])

In [ ]:
X_train_enc = pd.DataFrame(X_prep, columns=X_train['ordinal'].columns).astype('int')

Las varianzas asociadas al conjunto de características anterior viene dado por:

In [ ]:
X_train_enc.var()

Se importa el método de umbral de varianza y se aplica a las características del conjunto de datos

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=.47)
X_filtered = sel.fit_transform(X_train_enc)

In [ ]:
X_train_enc.shape

In [ ]:
X_filtered.shape

Con lo que se filtran 9 caracteristicas, para obtener las caraterísticas seleccionadas se hace uso del método `.get_support()` para obtener una mascara de los indices seleccionados

In [ ]:
X_train['ordinal'].columns[sel.get_support()]

### Métodos Univariados para Regresión 

Dentro de las técnicas clásicas de selección de variables en contextos de **regresión** encontramos los métodos:

**Input numérico u Ordinal**

* Coeficiente de correlación de Pearson (scoring lineal)

Este coeficiente mide la relación lineal entre dos variables aleatorias gaussinas y ha sido utilizado de manera frecuente durante el curso. Para seleccionar características utilizando este coeficiente se puede hacer uso de la función `f_regression` del módulo `feature_selection` de Scikit-learn.


* Coeficiente de Spearman (scoring no lineal) 

El coeficiente de correlación de Spearman corresponde a una medida de correlación de *rango*, esto se refiere a que permite cuantificar relaciones entre variables ordinales. En este caso, si se trabaja con variables numéricas, es necesario categorizarlas en intervalos ordenados para luego compararlas. La ventaja de esto es que no se requiere especificar distribuciones para los datos, por lo que es un test no paramétrico. Este coeficiente permite describir si la relación entre dos variables aleatorias es *monotona*.

Si se poseen $N$ $X_i$ e $Y_i$ de las variables aleatorias $X$ e $Y$, es posible calcular el coeficiente de correlación de Spearman $r_s$ generando transformaciones de rango $\pi_X$ y $\pi_Y$ sobre los valores $X_i$ e $Y_i$. Lo que esta transformación hace es ordenar las muestras asociadas a cada variable aleatoria asociandoles una posición.

**Ejemplo**

Se define una variable aleatoria y se calcula su transformación de rango.

In [ ]:
X = np.random.rand(10)

def pi_x(X):
    X_ = X.copy()
    X_.sort()
    return [*enumerate(X_,start=1)]

In [ ]:
pi_x(X)

Cuando dos o más valores se repiten, se debe definir una regla de desempate, esta consiste en asignar rangos promedio, es decir, si el valor $X_i$ se repite $l$ veces, entonces se les asigna el promedio de sus rangos, es decir $\pi_X(X_i^{(k)}) = \frac{1}{l}\sum_{j=1}^l \pi_X(X_i^{(j)})$ para todo $k = 1,...,l$ donde $X_i^{(k)}$ representa cada copia de $X_i$.

**Ejercicio**

1. Modifique la función de asignación de rangos para que implemente esta regla de desempate.

Haciendo uso de las transformaciones $\pi_X$ y $\pi_Y$ se calcula 
$r_s$ por medio de:
$$
r_{s}=\frac{\operatorname{cov}\left(\pi_{X}, \pi_{Y}\right)}{\sigma_{\pi_{X}} \sigma_{\pi_{Y}}}
$$
Donde $\sigma_{\pi_{X}}$ y $\sigma_{\pi_{Y}}$ corresponden a las desviaciones estándar sobre los rangos de las variables. El calculo anterior corresponde por tanto al coeficiente de correlación de Pearson sobre la categorización ordinal de las variables de interés. Para calcular el coeficiente de correlación de Spearman entre las variables de numéricas se puede hacer uso del método de pandas `.corr()`.

**Ejemplo**

Se calcula el coeficiente de correlación de Spearman sobre las variables numéricas

In [ ]:
spearman = df_train['numeric'].corr(method='spearman')

In [ ]:
spearman.nlargest(5,columns='SalePrice')['SalePrice']

In [ ]:
spearman.nsmallest(5,columns='SalePrice')['SalePrice']

un coeficiente de Spearman muy cercano a 1 o a -1 indica relaciones monotonas entre las variables, por otra parte un coeficiente cercano a 0 sugiere ausencia de tal tipo de relaciones. Con esto es posible hacer un filtrado de variables poniendo un umbral de correlación.

**Ejercicio**

1. La función `spearmanr` de `scipy.stats` permite tener acceso a un valor de significancia para contrarrestar la hipotesis nula _" No hay relación entre las caracteristicas"_. Seleccione ciertas variables numéricas y utilizando dicha función.

* Coeficiente de correlación de Kendall (scoring no lineal) 

Este coeficiente mide la *concordancia* entre los rangos  de dos variables aleatorias . Los rangos asociados a cada valor son calculados de la misma manera que para el coeficiente de correlación de Spearman. Si se obtienen dichos rangos, es posible calcular el *número de pares concordantes* $C$ y *número de pares no concordantes* $D$, para ello, si se poseen $N$ muestras $X_i$ e $Y_i$ de las variables aleatorias $X$ e $Y$, se generan sus transformaciones de rango correspondientes $x_i = \pi_X(X_i)$ y $y_i = \pi_Y(Y_i)$, luego se generan pares ordenados $(x_i, y_i)$. Para finalizar, el coeficiente de correlación $\tau$ de Kendall se obtiene mediante la formula:
$$
\tau=\frac{2}{n(n-1)} \sum_{i<j} \operatorname{sgn}\left(x_{i}-x_{j}\right) \operatorname{sgn}\left(y_{i}-y_{j}\right)
$$

**Ejemplo**

Se utiliza el coeficiente de correlación de Kendall con pandas.

In [ ]:
kendall = df_train['numeric'].corr(method='kendall')
kendall.nsmallest(5,columns='SalePrice')['SalePrice']

In [ ]:
kendall.nlargest(5,columns='SalePrice')['SalePrice']

En este caso, un valor cercano a 1 indica alta concordancia, cercano a -1 indica alta discordancia. Un valor cercano a 0 indica la ausencia de relación. 

**Ejercicios**

1. Utilice la función `kendalltau` del módulo `scipy.stats` para realizar pruebas de hipótesis sobre la relación entre variables numéricas y la variable predictiva.

2. Scikit-learn ofrece la función `f_regression` del módulo `feature_selection`. Esta permite realizar un test F univariado. Con este test se captura la dependencia lineal entre dos vectores. ¿Es posible interpretar el estadístico F como un puntaje de selección de variables? 

* Información Mutua

La información mutua entre dos variables $X$ e $Y$ busca medir dependecia. Así, si ambas son independientes, no existe información de $Y$ que se pueda obtener por medio de $X$ (lo mismo en el sentido contrario) luego el valor asociado de información mutua para estas variables será 0. Si por otra parte $X$ es una función determinista de $Y$ se tendrá un valor de información mutua máximo, en general, mayores valores de este valor indican mayor dependencia (puede ser usado como un scoring). En términos matemáticos, si $(X,Y)$ es un par de variables aleatorias, si su distribución conjunta se denota por $P_{(X,Y)}$ y sus marginales por $P_X$ y $P_Y$ se define la información mutua $I(\cdot,\cdot)$ como:

$$
I(X;Y) = D_{KL}(P_{(X,Y)} || P_X \otimes P_Y))
$$

Donde $D_{KL}$ es la divergencia de Kullback-Leibler. Esta se define para dos medidas de probabilidad $P, Q$ sobre $\mathcal{X}$, con $P$ absolutamente continua con respecto a $Q$ según la ecuación:
$$
D_{\mathrm{KL}}(P \| Q)=\int_{\mathcal{X}} \log \left(\frac{d P}{d Q}\right) d P
$$

En el caso discreto esto equivale a
$$
D_{\mathrm{KL}}(P \| Q)=\sum_{x \in \mathcal{X}} P(x) \log \left(\frac{P(x)}{Q(x)}\right)
$$
Con lo que la información mutua se puede escribir como:

$$
\mathrm{I}(X ; Y)=\sum_{y \in \mathcal{Y}} \sum_{x \in \mathcal{X}} P_{(X, Y)}(x, y) \log \left(\frac{P_{(X, Y)}(x, y)}{P_{X}(x) P_{Y}(y)}\right)
$$

La ventaja de esta medida de dependencia recae en que puede cuantificar relaciones de dependencia no lineales.

**Ejemplo**

Se aplica el criterio de información mutua para filtrar características, para ello se importa el módulo correspondiente

In [ ]:
from sklearn.feature_selection import mutual_info_regression

Y = df_train['numeric'].SalePrice.values.reshape([-1,])
X = df_train['numeric'].GrLivArea.values.reshape([-1,])

I = mutual_info_regression(df_train['numeric'].drop('SalePrice', axis=1),Y)
I

Se puede ver que hay variables indicadas como independientes de `SalePrice` por lo que puede ser beficioso para el modelo extraerlas.

**Ejercicios**

1. Genere un conjunto de datos `D` de 3 dimensiones y 1000 registros utilizando `np.random.rand`.

2. Genere una variable `y` dependiente de `D`, para ello, `y` debe depender de manera lineal de la primera columna de `D` y de manera no lineal de la segunda columna de `D` (use una transformación trigonométrica por ejemplo). Añada ruido aleatorio normal. 

3. Haga un test F para medir la relación entre las columnas de `D` (características) e `y`. 

4. Haga calcule la información mutual entre las características de `D` e `y`. Compare con los resultados de del test F. 

### Métodos Univariados para Clasificación

Cuando la variable de respuesta es discreta se pueden aplicar las siguientes técnicas.

**Input Númerico**

* Test F ANOVA 

Este método ya fue estudiado. Permite detectar si la variable de respuesta (discreta) genera grupos significativamente diferenciables en la variable numérica input. Scikit-learn ofrece la función `f_classif` del módulo `feature_selection` para realizar la versión one-way de este test. Observe que se puede utilizar para inputs numéricos en problemas de regresión. Este test requiere que los grupos inducidos por las categorías sean distribuidos de manera normal, lo cual dificilmente se cumple en la práctica.

* Información Mutua

Este método es aplicable a problemas de clasificación con inputs numéricos. 

**Inputs discretos**

* Test Chi cuadrado

El test de independencia $\chi^2$ permite determinar si existen relaciones de dependencia significativas entre dos variables discretas. El estadístico $\chi^2$ comprueba la existencia de diferencias significativas entre las frecuencias observadas y las frecuencias esperadas para las variables estudiadas. Acá la hipótesis nula es `No exsite una asociación entre las variables`. Para la construcción de este test se genera una tabla de contingencia, esta consiste en una matriz con entradas del tipo $O_{ij}$, cada una de las cuales representa la cantidad de veces que se observa la categoría (feature) $x_i$ versus la categoría $y_j$ (respuesta). De esta forma, se pueden obtener totales por fila $N_{i,.}$ y por columna $N_{.,j}$. Con estos valores se obtienen los *conteos esperados* $\hat{E}_{ij}$ por medio de:
$$
\hat{E}_{i, j}=\frac{\left(N_{i, .}\right)\left(N_{., j}\right)}{\sum_{i}N_{i,.} + \sum_{j}N_{.,j}}
$$

Así, el estadístico asociado a la relación de categorías $i,j$ viene dado por:

$$
\chi^{2}=\sum_{i, j} \frac{\left(O_{i j}-\hat{E}_{i, j}\right)^{2}}{\hat{E}_{i, j}}
$$

Tal valor permite hacer pruebas de hipótesis o ser considerado como un puntaje para selección de características.

**Ejemplo**

Se aplica el test chi cuadrado sobre las variables categóricas, para ello, se transforma la variable `SalePrice` a una variable ordinal y se procesa por medio de encoders de Scikit-learn

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Features
X_train = df_train['categorical'].copy()
X_train.dropna(how='all', inplace=True)

# Variable dependiente
y_train = df_train['numeric']['SalePrice'].copy()
y_train.dropna(how='all', inplace=True)
y_train = pd.cut(y_train, bins=5)
y_train = np.array(y_train).reshape([-1,1])

# Se preparan los datos
encoder_one_hot = OneHotEncoder(sparse=False)
encoder_ordinal = OrdinalEncoder()

X_cat_enc = encoder_one_hot.fit_transform(X_train)
X_cat_enc = pd.DataFrame(
    X_cat_enc, columns=encoder_one_hot.get_feature_names()).astype(int)

y_train_enc = encoder_ordinal.fit_transform(y_train)

Finalmente se obtiene un arreglo de puntajes y valores de significancia para cada categoría

In [ ]:
from sklearn.feature_selection import chi2

chi2(X_cat_enc,y_train_enc)

**Ejercicio**

1. El método de información mutua funciona para variables discretas en problemas de clasificación. Investigue como implementarlo en selección de características en tal contexto.

En general, los métodos univariados de selección de caractrísticas permiten realizar selecciones basándose en valores interpretables como *puntajes*. Si se ve este proceso de selección como un filtrado de caracteŕisticas, entonces es sencillo implementarlo dentro de una *pipeline* de preprocesamiento, para esto, Scikit-learn ofrece los objetos `SelectKBest` para filtrar características según un número de dimensiones objetivo, `SelectPercentile` que permite remover variables según un porcentaje objetivo de dimensiones.

**Ejemplo**

Se seleccionan 5 variables categóricas según el resultado del test chi cuadrado.


In [ ]:
from sklearn.feature_selection import SelectKBest

# Se entrena y transfroma el selector
selector = SelectKBest(
    k=5,
    score_func=chi2,
)
selected_data = selector.fit_transform(X_cat_enc, y_train_enc)

# Se obtienen las categorias
selected_cats = selector.get_support()
selected_cats = X_cat_enc.columns[selected_cats]

print(
    'Categorias seleccioandas: \n',
    *[str(i + 1) + '.- ' + str(h) + '\n' for i, h in enumerate(selected_cats)])

**Ejercicios**

1. Utilice `SelectPercentile` del módulo `feature_selection` para seleccionar el 10% de las categrías con mayor puntuación según el estadístico chi cuadrado.

2. Lo métodos de proporcion de falso positivo (FPR), proporción de descubrimientos falso (FDR) y error por familias (FWE) permiten seleccionar caracteríssticas utilizando un valor de significacia versus p-valores en tests estdísticos. Investigue su formulación y aplique selección de variables utilizando los métodos `SelectFpr`, `SelectFdr` y `SelectFwe` respectivamente.

### Métodos envolventes 

Esta clase métodos de selección de caráterísticas utilizan un estimador externo y utilizan estrategias de búsqueda sobre suconjuntos de variables optimizando las métricas de rendimiento del estimador externo

### Interludio: Métodos de Estimación basados en Árboles

Los métodos basados en arboles operan particionando el espacio de características en un conjunto de rectangulos, luego se ajusta un modelo simple (cómo una constante $c$) en cada uno de estos rectangulos, con el fin de predecir la respuesta de los datos agrupados en esta región del espacio.

En concreto este proceso se reduce a encontrar cuales son las variables que mejor segmentan el conjunto de datos (con una desigualdad simple), en el caso binario para la primera etapa del algoritmo el conjunto de entrenamiento original es particionado en 2 subconjuntos. Este proceso se repite de forma recursiva en cada una de las separaciones. De esta forma se segmenta el conjunto de entrenamiento quedando asociada cada muestra a un nodo terminal del árbol ensamblado.

Después del proceso de crecimiento del arbol recién descrito dependiendo del contexto del problema abordado, puede seguir una etapa de podado del arbol, lo cual permite manejar el tamaño y problemas con **sobre-ajuste** en el conjunto de entrenamiento.

#### CART: Arboles de Regresión y Clasificación

En primer lugar se aborda la construcción para el problema de regresión, luego se revisará un método para construir clasificadores modificando algunos aspectos de la construcción que antecede.

##### Regresión:

Sea un conjunto de datos $\{ (x_i,y_i) \}_{i=1}^N$, donde cada $x_i \in \mathbb{R}^p$ e $y_i \in \mathbb{R}$ corresponde a la respuesta. Se define $X = (x_1,\ldots, x_N)^T \in \mathbb{R}^{N \times p}$. Esto corresponde al escenario clásico que se da en un problema de regresión.

La idea del algoritmo es generar reglas de desición automáticas para segmentar los puntos del conjuntos de entrenamiento y la topología de las regiones de desición. Para este desarrollo usaremos un esquema de partición binario, es decir el árbol resultante será un árbol binario.

Supongamos que ya se tiene una partición con $M$ elementos $R_1, \ldots, R_M$; donde se modela una respuesta constante $c_m$ en cada región, es decir:

\begin{equation}
f(x) = \sum_{m=1}^M c_m I{(x \in R_m)}
\end{equation}

Donde $I$ corresponde a la función indicatríz.

Al adoptar como criterio de perdida la minimización de la suma de los errores cuadráticos ($\sum(y_i - f(x_i))^2)$ es simple demostrar que el $c_m$ óptimo en cada región $R_m$ corresponde al promedio entre las respuestas, es decir:

\begin{equation}
\hat{c}_m = \frac{1}{|R_m|}\sum_{i:x_i \in R_m} y_i
\end{equation}

Encontrar la mejor partición en término de los errores cuadráticos es computacionalmente costoso, incluso prohibitivo. Por esta razón se emplea un esquema de optimización **glotón**, partiendo con todos los datos, considerando una variable $j$ para partición y un punto $s$, se difinen los siguiente semi planos:

\begin{align}
R_1(j,s) &= \{ x \in X | x_j \geq s\} \\
R_2(j,s) &= \{ x \in X | x_j < s\}
\end{align}

Luego para encontrar las variables $j,s$, se resuelve el siguiente problema:

\begin{equation}
\min_{j,s} \left \{ \min_{c_1} \sum_{i:x_i \in R_1(j,s)} (y_i-c_1)^2 + \min_{c_2} \sum_{i:x_i \in R_2(j,s)} (y_i-c_2)^2 \right \}
\end{equation}

para cualquier $j,s$ la elección óptima de $\hat{c}_1$ y $\hat{c}_2$ corresponderá al promedio los $y_i$ en las regiones $R_1$ y $R_2$ respctivamente.

Considerando que para cada $j$ variable de segmentación es facil determinar el punto $s$ óptimo. Es factible recorrer todas las características para encontrar el par óptimo $(j,s)$.

Habiendo obtenido el primer corte de los datos $R_1$ y $R_2$, el proceso de segmentación se realiza de forma recursiva en cada una de las regiones encontradas de forma sucesiva.

La pregunta que aparece de forma natural en esta etapa del proceso es que tan profundo debe ser extendido el arbol. Claramente una arbol demasiado grande puede conllevar a un **sobre-ajuste** del conjunto de entrenamiento, generando malas propiedades de generalización del modelo.

La estrategía que suele ser empleada es definir un parámetro $n$ que representa el tamaño mínimo que puede tener una hoja del arbol. De esta forma el arbol crece hasta que cada hoja de este tiene un tamaño a lo más $n$. Este arbol producto del proceso de separar sucesivamente los datos se define cómo $T_0$.

Para evitar lo mencionado en el parrafo anterior despues de crecer el arbol, se lleva a cabo un proceso de poda, es decir colapsar nodos del arbol (no terminales, hojas). Esto será explicado y se realiza en base a un criterio que considera la función de costo y la complejidad del arbol (tamaño), un criterio de **costo-complejidad**.

Se define el sub arbol $T \subset T_0$ cómo cualquier arbol que puede ser obtenido a partir de $T_0$ colapsando cualquier cantidad de nodos internos (no terminales) del arbol. Se indexan los nodos terminales de $T$ por $m$, cada uno representando una región $R_m$, adicionalmente $|T|$ representa la cantidad de nodos terminales de $T$.

Se define:

\begin{align}
N_m =& |R_m|\\
\hat{x}_m =& \frac{1}{N_m} \sum_{i:x_i \in R_m} y_i \\
Q_m(T) =& \frac{1}{N_m} \sum_{i:x_i \in R_m} (y_i - \hat{c}_m)^2
\end{align}

Con esto se define el criterio de **costo-complejidad** cómo:

\begin{equation}
C_{\alpha}(T) = \sum_{m=1}^{|T|} N_m Q_m(T) + \alpha |T| 
\end{equation}

La idea es encontrar para cada $\alpha$ el sub arbol $T_\alpha \subset T_0$ que minimice $C_\alpha(T)$. Se observa que $\alpha \geq 0$ funciona cómo un termino de penalización sobre el tamaño del arbol, valores grandes de $\alpha$ inducen arboles más pequeños. Si $\alpha= 0$ entonces se recupera $T_0$.

Para cada $\alpha$ se puede demostrar que existe un único $T_\alpha$ de tamaño mínimo que minimiza $C_\alpha$. Para encontrar $T_\alpha$ se usa la estrategía de podar el nodo que produce el menor incremento posible en $\sum_{m=1}^{|T|} N_m Q_m(T)$ hasta llegar a la raíz. Esto genera una secuencia finita de sub arboles, es posible demostrar que esta secuencia contine necesariamente a $T_\alpha$.

Para estimar el valor de $\alpha$ es posible recurrir a un esquema de validación cruzada (se recomienda 5-10) sobre un conjunto de validación. Finalmente se elige $\hat{\alpha}$ que minimíce la suma de errores cuadráticos en el esquema de validación cruzada. Finalmente el arbol óptimo será $T_\hat{\alpha}$.

La ventajas de los este método se consisten en su capacidad para manejar variables numéricas y categóricas de manera nativa. Además requieren de un procesamiento casi nulo de los datos, son además modelos no paramétricos y se comporta bien en altas dimensiones, 

#### Clasificación

Las definiciones para el problema de regresión fueron extensas, pero el lado positivo es que para abordar el problema de clasificación el esquema es homólogo solamente se deben ajustar los criterios de separación de nodos y el podado de este.

Cómo se trata de un problema de clasificación, los valores de la respuesta $y_i$ toma valores en un conjuntos discreto $1,\ldots, K$.

En el problema anterior se tomó como criterio de **pureza** dentro de los nodos la suma de errores cuadráticos, esto no es apropiado para clasificación.

En un nodo $m$ del arbol, que representa la región $R_m$ con $N_m$ observaciones, se define:

\begin{equation}
\hat{p}_{mk} = \frac{1}{N_m} \sum_{i: x_i \in R_m} I(y_i = k)
\end{equation}

como la proporción de observaciones de la clase $k$ en el nodo $m$. Se clasifican los puntos en el nodo $m$ según:

\begin{equation}
k(m) = \arg \max_k \hat{p}_{mk}
\end{equation}

es decir, se asignan los puntos en el nodo $m$ a la clase que mayor presencia tenga en el nodo.

Para el problema de clasificación se suele usar distintas forma de $Q_m(T)$ (pureza del nodo), los más comones son:

- **Error de clasificación:**
\begin{align}
Q_m^1(T) =& \frac{1}{N_m} \sum{i \in R_m} I(y_i \neq k(m))\\
=& 1- \hat{p}_{mk(m)}
\end{align}

- **Coeficiente de Gini:**
\begin{align}
Q_m^2(T) =& \sum_{k \neq k'} \hat{p}_{mk} \hat{p}_{mk'}\\
=& \sum_{k=1}^K \hat{p}_{mk}(1 - \hat{p}_{mk})\\
\end{align}

- **Entropía:**
\begin{equation}
Q_m^3(T) = -\sum_{k=1}^K \hat{p}_{mk}\log\hat{p}_{mk}
\end{equation}

Las ventajas de $Q_m^2$ y $Q_m^3$ sobre $Q_m^1$, es que estos son diferenciables, lo cual puede ser fundamental para esquemas de optimización numéricos.

Cabe mencionar que que los semi planos utilizados para generar las bifurcaciones en cada nodo del arbol no es necesario que sean de la forma $\{ x \in X | x_j \geq s\}$, tambien es posible considerar combinaciones lineales de las características. Esto por lo general puede mejorar la precisión pero se pierde por otro lado interpretabilidad del modelo.

## RFE
## Reducción de Dimensionalidad
## Métodos Básicos de Estimación
## Selección de Caracterísitcas implicita

